In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/pandhari/Desktop/MLOPs/End-To_End_Projects/Text_Summariser'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs : int
    warmmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int


In [7]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml , create_directories

In [8]:
class ConfigurationManager:
    def __init__(self , config_filepath = CONFIG_FILE_PATH , params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
 

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir , 
            data_path=config.data_path , 
            model_ckpt=config.model_ckpt , 
            num_train_epochs=params.num_train_epochs , 
            warmmup_steps=params.warmmup_steps , 
            per_device_train_batch_size=params.per_device_train_batch_size , 
            weight_decay=params.weight_decay , 
            logging_steps=params.logging_steps , 
            evaluation_strategy=params.evaluation_strategy , 
            eval_steps=params.eval_steps , 
            save_steps=params.save_steps , 
            gradient_accumulation_steps=params.gradient_accumulation_steps  
        )

        return model_trainer_config

    

In [12]:
from transformers import AutoModelForSeq2SeqLM , AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer
import torch
from datasets import load_from_disk

[2024-12-10 22:40:29,122: INFO: config: PyTorch version 2.5.1 available.]


In [13]:
class ModelTrainer:
    def __init__(self , config:ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)


        # Loading the data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments (
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6, gradient_accumulation_steps=16
        )

        trainer = Trainer ( model=model_pegasus, args=trainer_args,
                    tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                    train_dataset=dataset_samsum_pt["test"],
                    eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        # Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir , "pegasus-samsum-model"))


        # Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir , "tokenizer"))




In [14]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate


Found existing installation: transformers 4.47.0
Uninstalling transformers-4.47.0:
  Successfully uninstalled transformers-4.47.0
Found existing installation: accelerate 1.2.0
Uninstalling accelerate-1.2.0:
  Successfully uninstalled accelerate-1.2.0
  Using cached transformers-4.47.0-py3-none-any.whl.metadata (43 kB)
  Using cached accelerate-1.2.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.47.0-py3-none-any.whl (10.1 MB)
Using cached accelerate-1.2.0-py3-none-any.whl (336 kB)


In [15]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()

[2024-12-10 22:40:41,228: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-12-10 22:40:41,230: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-10 22:40:41,231: INFO: common: created directory at : artifacts]
[2024-12-10 22:40:41,231: INFO: common: created directory at : artifacts/model_trainer]


ChunkedEncodingError: ('Connection broken: IncompleteRead(107309772 bytes read, 825840831 more expected)', IncompleteRead(107309772 bytes read, 825840831 more expected))